In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import numpy as np
from rank_bm25 import BM25Okapi
import pickle as pkl
import pandas as pd
from tqdm.auto import tqdm
import ast


/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [83]:
with open('chat_gpt_iab.pkl', 'rb') as f:
    IAB_KG=pkl.load(f)

In [84]:
IAB_KG

[{'Graduation': ['Graduation ceremony',
   'Graduation cap and gown',
   'Diploma',
   'Commencement',
   'Graduation party',
   'Graduation speech',
   'Graduation announcement',
   'Graduation gifts',
   'Graduation photos',
   'Graduation date',
   'Graduation requirements',
   'Graduation invitation',
   'Graduation honors',
   'Graduation decorations',
   'Graduation year']},
 {'Shopping': ['Online shopping',
   'Retail stores',
   'E',
   'Shopping malls',
   'Discounts',
   'Coupons',
   'Deals',
   'Sales',
   'Loyalty programs',
   'Shopping carts',
   'Payment methods',
   'Product reviews',
   'Customer service',
   'Returns',
   'Exchanges',
   'Fashion',
   'Electronics',
   'Home goods',
   'Groceries',
   'Accessories',
   'Beauty products',
   'Gifts',
   'Holiday shopping',
   'Black Friday',
   'Cyber Monday',
   'Prime Day.']},
 {'Automotive': ['Car',
   'Vehicle',
   'Automobile',
   'Engine',
   'Transmission',
   'Suspension',
   'Brakes',
   'Tires',
   'Fuel eff

In [85]:
with open('chat_gpt_maf.pkl', 'rb') as f:
    MAF_KG=pkl.load(f)

In [86]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [87]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [88]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [89]:
maf=df_MAF['item'].to_list()

In [90]:
call=iab_labels+maf

In [91]:
#create doc query
expanded_call=[]

for items in tqdm(iab_labels):
    for dic in IAB_KG:
        key=list(dic.keys())[0]
        if items==key:
            list_items=dic[key]


    string_list=" ".join(list_items)
    string_list=string_list+' '+items
    string_list=string_list.replace("_", " ")
    expanded_call.append(string_list)

100%|██████████| 160/160 [00:00<00:00, 14086.96it/s]


In [92]:
for items in tqdm(maf):
    for dic in MAF_KG:
        key=list(dic.keys())[0]
        if items==key:
            list_items=dic[key]


    string_list=" ".join(list_items)
    string_list=string_list+' '+ items
    string_list=string_list.replace("_", " ")

    expanded_call.append(string_list)

100%|██████████| 942/942 [00:00<00:00, 4435.96it/s]


In [93]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/haskar140/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haskar140/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
def tokenizer(string, reg="[a-zA-Z'-]+|[0-9]{1,}%|[0-9]{1,}\.[0-9]{1,}%|\d+\.\d+%}"):
    regex=reg
    string=string.replace("-"," ")
    return " ".join(re.findall(regex, string))
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stem the tokens
    porter_stemmer = PorterStemmer()
    tokens = [porter_stemmer.stem(token) for token in tokens]
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    preprocessed_text = tokenizer(preprocessed_text)
    
    return preprocessed_text

In [95]:
iab_expanded=expanded_call[0:160]

In [96]:
maf_expanded=expanded_call[160:]

In [97]:
corpus_clean = [preprocess_text(x) for x in iab_expanded]

In [98]:
tokenized_corpus = [doc.split(" ") for doc in corpus_clean]
print(tokenized_corpus[0])
tokenized_corpus = [list(set(doc)) for doc in tokenized_corpus]
print(tokenized_corpus[0])
bm25 = BM25Okapi(tokenized_corpus)


['shampoo', 'condition', 'hair', 'oil', 'hair', 'serum', 'hair', 'mask', 'hair', 'treatment', 'hair', 'growth', 'hair', 'textur', 'hair', 'style', 'hair', 'tool', 'hair', 'color', 'hair', 'extens', 'scalp', 'care', 'dandruff', 'frizz', 'control', 'heat', 'protect', 'split', 'end', 'hair', 'loss', 'hair', 'vitamin', 'natur', 'hair', 'care', 'profession', 'hair', 'care', 'hair', 'care']
['tool', 'treatment', 'hair', 'protect', 'growth', 'dandruff', 'oil', 'profession', 'end', 'textur', 'style', 'control', 'condition', 'vitamin', 'frizz', 'shampoo', 'natur', 'serum', 'heat', 'color', 'split', 'extens', 'scalp', 'loss', 'care', 'mask']


In [99]:
def get_bm25_search_hits(query, chunks):
    tokenized_query = preprocess_text(query).split()
    tokenized_query=list(set(tokenized_query))
    sorted=np.sort(bm25.get_scores(tokenized_query), axis=0)[::-1]
    sorted=sorted[sorted>0]
    n=len(sorted)
    print(n)
    if n==0:
        return ["No Pred"]
    top_n = np.argsort(bm25.get_scores(tokenized_query), axis=0)[::-1]
    bm25_search = []
    for idx in top_n:
        if (len(bm25_search) < n):
    #         print(chunks[idx])
    #         print("-----------")
            bm25_search.append(chunks[idx])
        
    return bm25_search

In [100]:
maf_expanded[704]

'Pizza Italian cuisine Cheese Tomato sauce Dough Toppings Pepperoni Margherita Thin crust Deep dish Neapolitan Oven Delivery Pizzeria Slices Garlic Mushrooms Olives Sausage Pineapple Anchovies Bell peppers Onions Feta Mozzarella Parmesan Pesto BBQ sauce Buffalo chicken Vegetarian Gluten Calzone Stromboli Brick oven Wood Gourmet Fast food Take Leftovers Party food Comfort food Snack yellow lady-slipper'

In [101]:
tokenized_query = preprocess_text(maf_expanded[704]).split()
print(tokenized_query)

['pizza', 'italian', 'cuisin', 'chees', 'tomato', 'sauc', 'dough', 'top', 'pepperoni', 'margherita', 'thin', 'crust', 'deep', 'dish', 'neapolitan', 'oven', 'deliveri', 'pizzeria', 'slice', 'garlic', 'mushroom', 'oliv', 'sausag', 'pineappl', 'anchovi', 'bell', 'pepper', 'onion', 'feta', 'mozzarella', 'parmesan', 'pesto', 'bbq', 'sauc', 'buffalo', 'chicken', 'vegetarian', 'gluten', 'calzon', 'stromboli', 'brick', 'oven', 'wood', 'gourmet', 'fast', 'food', 'take', 'leftov', 'parti', 'food', 'comfort', 'food', 'snack', 'yellow', 'lady', 'slipp']


In [102]:
print(get_bm25_search_hits(maf_expanded[704],iab_labels))

50
['Food & Drink', 'Barbecues and Grilling', 'World Cuisines', 'Dining Out', 'Household Supplies', 'Food Industry', 'Grocery Shopping', 'Home Appliances', 'Desktops', 'Large Animals', 'Bars & Restaurants', 'Cooking', 'Retail Industry', 'Musical Instruments', 'Cinemas and Events', 'Home Entertaining', 'Convertible', "Men's Casual Wear", 'Minivan', 'Luxury Cars', 'Graduation', "Men's Formal Wear", "Women's Handbags and Wallets", 'Commercial Trucks', 'Road-Side Assistance', 'Birthday', "Women's Shoes and Footwear", 'Games and Puzzles', 'Europe Travel', 'Flower Shopping', 'Fashion Events', 'Africa Travel', "Women's Clothing", 'North America Travel', 'South America Travel', 'Personal Celebrations & Life Events', 'Wedding', 'Smartphones', 'Board Games and Puzzles', 'Pets', 'Asia Travel', 'Beach Travel', 'Travel', 'Events and Attractions', 'Travel Locations', 'Holiday Shopping', 'Malls & Shopping Centers', 'Hospitality Industry', "Children's Games and Toys", 'Australia and Oceania Travel']


In [103]:
nodes=df_MAF['item'].to_list()
node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [104]:
top_n_labels=[]
for items in tqdm(maf_expanded):
    top_n_labels.append(get_bm25_search_hits(items,iab_labels))

    

  5%|▍         | 47/942 [00:00<00:03, 245.07it/s]

88
53
58
58
95
95
96
96
95
98
95
96
96
96
95
95
67
100
101
59
51
46
46
47
50
77
80
77
77
29
36
31
35
47
48
61
64
62
44
26
38
29
80
19
19
42
43
77
28
77
50
54
50


  8%|▊         | 72/942 [00:00<00:04, 210.76it/s]

65
65
54
55
23
25
39
23
23
26
25
56
94
97
47
79
80
57
57
57
45
54
45
63
42
90
90
56
25
27
28
27
26


 12%|█▏        | 114/942 [00:00<00:04, 179.72it/s]

...

59
59
33
13
26
29
26
27
30
26
58
60
58
60
59
62
58
36
36
37
38
37
48
48
51
56
56
56
66
21
37
48
11
32
108
108
108
96
20
20
25
33


 95%|█████████▍| 894/942 [00:04<00:00, 239.14it/s]

21
20
71
87
87
46
46
61
11
43
43
31
59
31
40
60
59
40
40
51
22
23
24
11
27
87
69
70
32
33
37
38
40
73
38
50
62
35
36
38
36
98
99
45
44
56
28
45
49
50
45
46
64
29
12
93


100%|██████████| 942/942 [00:04<00:00, 220.93it/s]

39
40
46
42
80
48
49
40
60
53
53
20
20
24
79
81
52
69
49
50
49
63
64
38
38
38
42
66
67
51


In [105]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [106]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [107]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [108]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    print("Count {}".format(count))
    print(proper)
    print(labels)



    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f

   

    candidate_count+=len(proper)
    for items in labels:
        if items in proper:
            hit_count+=1
    print("C Count {}".format(candidate_count))
    print(hit_count)
    # if count==20:
    #     break


Count 1
['Style & Fashion', "Women's Clothing", "Men's Clothing", "Men's Formal Wear"]
['Home Improvement', "Men's Casual Wear", 'Remodeling & Construction', 'Automotive', 'Home Entertaining', 'Minivan', 'Home Security', 'Gardening', 'Van', 'Landscaping', "Women's Shoes and Footwear", 'SUV', 'Home & Garden', 'Automotive Industry', 'Green Vehicles', 'Style & Fashion', 'Auto Parts', 'Makeup and Accessories', 'Aviation Industry', "Women's Jewelry and Watches", 'Auto Repair', 'Houses', "Women's Formal Wear", 'Bath and Shower', "Men's Clothing", "Women's Clothing", 'Office Property', 'Apparel Industry', 'Barbecues and Grilling', 'Space and Astronomy', 'Tablets and E-readers', 'Rail Travel', 'Flower Shopping', 'Retail Industry', 'Smart Home', 'Education industry', 'Theater Venues and Events', 'Concerts & Music Events', 'Shopping', 'Hotel Properties', 'Motorcycles', 'Fitness and Exercise', "Women's Handbags and Wallets", 'Smartphones', 'Video Gaming', 'Pickup Trucks', 'Shaving', 'Outdoor Acti

In [109]:
nodes[19]

'residence'

In [110]:
MAF_predictions[0]

'[\'Style & Fashion\', "Women\'s Clothing", "Men\'s Clothing", "Men\'s Formal Wear"]'

In [111]:
"Men's Formal Wear" in top_n_labels[0]

True

In [112]:
hit_count/candidate_count

0.6544284632853898

In [113]:
precision/942

0.038921501775478366

In [114]:
recall/942

0.05028663414330074

In [115]:
f1/942

0.031960911340076115